# 3.6 层级索引

In [1]:
import pandas as pd
import numpy as np

## 3.6.1 多层级索引Series

### 1. 笨办法

In [2]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), 
         ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010): ('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### 2. 好方法: Pandas多级索引

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### 3. 高维数据的多级索引

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [10]:
pop_df = pd.DataFrame({
    'total': pop,
    'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]
})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## 3.6.2 多级索引的创建方法

In [12]:
df = pd.DataFrame(np.random.rand(4, 2), 
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.297386  0.210687
  2  0.818327  0.890146
b 1  0.044466  0.662138
  2  0.034878  0.791117

In [13]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### 1. 显式地创建多级索引

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [17]:
# labels -> codes
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

/home/cyx/code/DataScienceHandbook/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  


MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

### 2. 多级索引的等级名称

In [18]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 3. 多级列索引

In [19]:
# 多级行列索引
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                  names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names=['subject', 'type'])

# 模拟数据
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# 创建DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  36.2  40.0  35.8  29.0  36.9
     2      41.0  36.8  33.0  36.1  35.0  36.0
2014 1      38.0  36.7  37.0  36.8  31.0  37.7
     2      48.0  37.8  31.0  35.1  31.0  35.9

In [20]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      40.0  35.8
     2      33.0  36.1
2014 1      37.0  36.8
     2      31.0  35.1

## 3.6.3 多级索引的取值与切片

### 1. Series多级索引

In [21]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [22]:
pop['California', 2000]

33871648

In [23]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [24]:
pop.loc['California': 'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [25]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [26]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [27]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### 2. DataFrame多级索引

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  36.2  40.0  35.8  29.0  36.9
     2      41.0  36.8  33.0  36.1  35.0  36.0
2014 1      38.0  36.7  37.0  36.8  31.0  37.7
     2      48.0  37.8  31.0  35.1  31.0  35.9

In [29]:
health_data['Guido', 'HR']

year  visit
2013  1        40.0
      2        33.0
2014  1        37.0
      2        31.0
Name: (Guido, HR), dtype: float64

In [30]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      32.0  36.2
     2      41.0  36.8

In [31]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        32.0
      2        41.0
2014  1        38.0
      2        48.0
Name: (Bob, HR), dtype: float64

In [32]:
# 元祖中使用切片导致语法异常
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-32-ec84d98d7d08>, line 2)

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,32.0,40.0,29.0
2014,1,38.0,37.0,31.0


## 3.6.4 多级索引行列转换

### 1. 有序的索引与无序的索引

In [34]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.201647
      2      0.058320
c     1      0.761343
      2      0.908369
b     1      0.242769
      2      0.407240
dtype: float64

In [35]:
try:
    data['a': 'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [36]:
data = data.sort_index()
data

char  int
a     1      0.201647
      2      0.058320
b     1      0.242769
      2      0.407240
c     1      0.761343
      2      0.908369
dtype: float64

In [37]:
data['a': 'b']

char  int
a     1      0.201647
      2      0.058320
b     1      0.242769
      2      0.407240
dtype: float64

### 2. 索引stack与unstack

In [38]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [39]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [40]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 3. 索引的设置与重置

In [41]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [42]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## 3.6.5 多级索引的数据累计方法

In [43]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  36.2  40.0  35.8  29.0  36.9
     2      41.0  36.8  33.0  36.1  35.0  36.0
2014 1      38.0  36.7  37.0  36.8  31.0  37.7
     2      48.0  37.8  31.0  35.1  31.0  35.9

In [44]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     36.5  36.50  36.5  35.95  32.0  36.45
2014     43.0  37.25  34.0  35.95  31.0  36.80

In [45]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,35.0,36.300000
2014,36.0,36.666667
